In [ ]:
import torch, torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.utils
from alexnet import AlexNet
import torch.nn as nn
from torch.optim import Adam


In [ ]:
W = 67 # minimum width for input to pass through alexnet
transform = transforms.Compose([
    transforms.Resize((W, W)), # the original mnist image size is 28*28, alexnet input size is 227*227
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

train = MNIST(root='./data', train=True, download=True, transform=transform)
test = MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
batch_size = 64

def transform_y(batch):
    imgs = []
    labels = []
    for idx in range(len(batch)):
        img = batch[idx][0]
        imgs.append(img)
        label_onehot = torch.zeros(10)
        label_onehot[batch[idx][1]] = 1
        labels.append(label_onehot.unsqueeze(0))

    imgs, labels = torch.concat(imgs, 0), torch.concat(labels, 0)
    return imgs.unsqueeze(1), labels

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, collate_fn=transform_y)
test_loader = torch.utils.data.DataLoader(test, batch_size, shuffle=False, collate_fn=transform_y)

model = AlexNet(10, 1)

In [ ]:
learning_rate = 0.01
num_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        print(labels.shape, outputs.shape, images.shape)
        # print(outputs[0], labels)
        break
        loss = criterion(outputs, labels)

        # backward and optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    break

    print("epoch:" + str(epoch) + "loss:" + str(loss))



In [ ]:
%debug